In [1]:
# Gerekli kütüphaneleri import edelim
import pandas as pd
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

# Veriyi yükleyelim
file_path = 'veriseti.csv'
df = pd.read_csv(file_path)

# Zero-shot sınıflandırma pipeline'ını yükleyelim
classifier = pipeline("zero-shot-classification")

# Sınıflandırmak için etiketler (pozitif, negatif, nötr)
candidate_labels = ["pozitif", "negatif", "nötr"]

# Yorum metinlerini alalım
reviews = df['Lemmatized_Review'].tolist()

# Her yorum için zero-shot sınıflandırma yapalım
results = []
for review in reviews:
    result = classifier(review, candidate_labels)
    results.append(result)

# Zero-shot modelinden elde ettiğimiz etiketleri ve skorları dataframe'e ekleyelim
df['Predicted_Label'] = [result['labels'][0] for result in results]
df['Predicted_Score'] = [result['scores'][0] for result in results]

# Özellik çıkarımı için TF-IDF kullanacağız
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['Lemmatized_Review'])

# Duygu etiketlerini sayısal değerlere dönüştürme (etiketler: 0 - negatif, 1 - nötr, 2 - pozitif)
label_map = {"negatif": 0, "nötr": 1, "pozitif": 2}
df['Label_Numeric'] = df['Predicted_Label'].map(label_map)

# Eğitim ve test verilerini ayıralım
X_train, X_test, y_train, y_test = train_test_split(X, df['Label_Numeric'], test_size=0.2, random_state=42)

# SVM modelini eğitelim
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

# Test verisi üzerinde tahmin yapalım
svm_predictions = svm.predict(X_test)

# Sonuçları değerlendirelim
print("SVM Modeli Doğruluk Skoru:", accuracy_score(y_test, svm_predictions))

# Sonuçları gösterelim
df.head()


No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


SVM Modeli Doğruluk Skoru: 0.5424063116370809


,Comment_ID,Lemmatized_Review,Predicted_Label,Predicted_Score,Label_Numeric
0,1,update video form,nötr,0.445382,1
1,2,call name cum peach walked parasite killer pea...,pozitif,0.496082,2
2,3,another bong hit,nötr,0.491303,1
3,4,expectation high holy fuck,nötr,0.497351,1
4,5,question people rate 45 want literally want,nötr,0.419980,1


In [6]:
# Gerekli kütüphaneleri yükleyin
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

# Veri setini yükleyin
file_path = 'auto_labeled_reviews.csv'
df = pd.read_csv(file_path)

# Label'ları sayısal değerlere dönüştürme (etiketler: 0 - Negatif, 1 - Nötr, 2 - Pozitif)
label_map = {"Negatif": 0, "Nötr": 1, "Pozitif": 2}
df['Label_Numeric'] = df['Label'].map(label_map)

# Veri setini eğitim ve test olarak ayıralım
X_train, X_test, y_train, y_test = train_test_split(df['Lemmatized_Review'], df['Label_Numeric'], test_size=0.2, random_state=42)

# TF-IDF kullanarak özellik çıkarımı
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# SVM ile model oluşturma
svm = SVC(kernel='linear')
svm.fit(X_train_tfidf, y_train)

# SVM ile tahmin yapma
svm_predictions = svm.predict(X_test_tfidf)
print("SVM Accuracy:", accuracy_score(y_test, svm_predictions))

# BERT modelini kullanabilmek için Hugging Face kütüphanesini yükleyin
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# BERT için veriyi hazırlayın
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=128)

# BERT modelini eğitmek için veri sınıflandırıcıları
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, y_train.tolist())
test_dataset = SentimentDataset(test_encodings, y_test.tolist())

# Eğitim parametrelerini ayarlayın
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
    report_to=None  # W&B kullanımı iptal edilir
)


trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Modeli eğitin
trainer.train()

# Model ile tahmin yapma
bert_predictions = trainer.predict(test_dataset)
bert_pred_labels = np.argmax(bert_predictions.predictions, axis=1)
print("BERT Accuracy:", accuracy_score(y_test, bert_pred_labels))


SVM Accuracy: 0.7396449704142012


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,1.008700
20,0.846600
30,0.818200
40,0.938900
50,0.781900
60,0.739000
70,0.792700
80,0.598200
90,0.652400
100,0.744000


BERT Accuracy: 0.8579881656804734
